### Pokemon Data Analytics

In [68]:
## importing libraries 
import numpy as np
import pandas as pd

## Loading our data
## not setting # as default index because of repetitions
df = pd.read_csv('../datasets/Pokemon.csv')#, index_col = 0)
# df.shape
df.head()
# df.sample(5)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


## Step 1 - Exploring the data

In [45]:
## longest name vs shortest
print("Longest name vs shortest : {} - [{}] vs. {} - [{}]".format(max(df.Name), len(max(df.Name)), min(df.Name), len(min(df.Name))))


Longest name vs shortest : Zygarde50% Forme - [16] vs. Abomasnow - [9]


# There seems to be an error
When we first ran this, we had shellos (below) as part of the sample. 
Abomasnow has a length of 9, whilst Shellos had 7

In [3]:
## Potential candidate for shortest :
df[df.Name == 'Shellos']

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
469,422,Shellos,Water,NaN,325,76,48,48,57,62,34,4,False


In [13]:
# even the following; there's over 300 names shorter than length of 9
# df[df['Name'].str.len()  < 8]

# and even with max; there's plenty of names longer than 16 characters
df[df['Name'].str.len() > 16]

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
7,6,CharizardMega Charizard X,Fire,Dragon,634,78,130,111,130,85,100,1,False
8,6,CharizardMega Charizard Y,Fire,Flying,634,78,104,78,159,115,100,1,False
12,9,BlastoiseMega Blastoise,Water,NaN,630,79,103,120,135,115,78,1,False
19,15,BeedrillMega Beedrill,Bug,Poison,495,65,150,40,15,80,145,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
786,711,GourgeistLarge Size,Ghost,Grass,494,75,95,122,58,75,69,6,False
787,711,GourgeistSuper Size,Ghost,Grass,494,85,100,122,58,75,54,6,False
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True


In [56]:
## sourced : https://stackoverflow.com/questions/21295334/find-length-of-longest-string-in-pandas-dataframe-column
df.Name.str.len() == df.Name.map(len).max()

## double checking
## print(len(df.iloc[7]) == len(df.iloc[124]))

0      False
1      False
2      False
3      False
4      False
       ...  
795    False
796    False
797    False
798    False
799    False
Name: Name, Length: 800, dtype: bool

In [18]:
## Getting column names
columns = list(df.columns)
# print(*columns, sep = ' - ')

## Loading specific items
df.iloc[[798,786,12,329,794], [1]]

,Name
798,HoopaHoopa Unbound
786,GourgeistLarge Size
12,BlastoiseMega Blastoise
329,MawileMega Mawile
794,Zygarde50% Forme


In [12]:
## Understanding objects
## 1 - type() (what is this thing?
## 2 - dir() (what can I do with it?)
## 3 - help() (tell me more)

# data - HoopaHoopa Unbound ; GourgeistLarge Size ; BlastoiseMega Blastoise ; MawileMega Mawile